In [241]:
import glob
from os.path import join, isfile, isdir, basename, split, splitext, exists
from os import makedirs, rename
import numpy as np
import pandas as pd
import numpy.matlib
import cv2
import shutil
import copy
from collections import defaultdict
import distinctipy
from itertools import combinations_with_replacement
from matplotlib import pyplot as plt
from matplotlib.font_manager import findfont, FontProperties
from skimage.measure import find_contours, regionprops
from skimage.draw import line, ellipse_perimeter
from skimage.morphology import dilation, square
from scipy.spatial.distance import cdist


In [41]:
# Return all file names at a given path
def get_all_files(path, only_filenames, sorted_list):
    file_list = [x for x in glob.glob(join(path, '*.*')) if isfile(x)]
    if only_filenames:
        file_list = [x.split('\\')[-1] for x in file_list]
    if sorted_list:
        return sorted(file_list)
    return file_list

# Return all folder names at a given path
def get_all_folders(path, only_foldernames, sorted_list):
    file_list = [x for x in glob.glob(join(path, '*')) if isdir(x)]
    if only_foldernames:
        file_list = [x.split('\\')[-1] for x in file_list]
    if sorted_list:
        return sorted(file_list)
    return file_list


In [42]:
# Read mask saved as RGB and flatten it to 2D
def read_mask(mask_path):
    init_mask = cv2.imread(mask_path)
    if len(init_mask.shape) == 2:
        return init_mask
    return np.max(init_mask, axis=2)

In [43]:
# Compute the intersection between an infinite line and a limited segment
# Points a and b define the line, points c and d define the segment
def line_segment_intersection(a, b, c, d):
    # The denominator comes from a special mathematical formula
    denominator = (a[0] - b[0]) * (c[1] - d[1]) - (a[1] - b[1]) * (c[0] - d[0])
    if denominator == 0:
        # The segment is parallel with the line. Return error.
        return False, None
    
    # Parametric values of the intersection:
    # t is between 0 and 1 if the intersection falls between the two points (a and b) of the line
    # u is between 0 and 1 if the intersection falls between the two points (c and d) of the segment
    t = ((a[0] - c[0]) * (c[1] - d[1]) - (a[1] - c[1]) * (c[0] - d[0])) / denominator
    u = ((a[0] - c[0]) * (a[1] - b[1]) - (a[1] - c[1]) * (a[0] - b[0])) / denominator

    # Check if lines actually intersect
    if (0 <= u <= 1):
        # Intersection falls inside the segment
        return True, np.array([a[0] + t * (b[0] - a[0]), a[1] + t * (b[1] - a[1])], dtype=float).reshape((1, 2))
    
    # Intersection falls outside the segment
    return False, None



# Compute the intersection between an infinite line (defined by two points) and a contour (a list of at least two points)
def line_contour_intersection(line, contour):
    intersection_pt = None
    min_intersection_dist = np.inf
    
    # Compute intersection of the line with each segment of the contour
    for seg_idx in range(contour.shape[0] - 1):
        intersection_found, curr_intersection_pt = line_segment_intersection(
            line[0, :], line[1, :], 
            contour[seg_idx, :], contour[seg_idx + 1, :])

        if intersection_found:
            curr_dist = cdist(line[:1, :], curr_intersection_pt)
            if curr_dist < min_intersection_dist:
                # Keep only the closest intersection
                intersection_pt = curr_intersection_pt
                min_intersection_dist = curr_dist
    return intersection_pt


In [130]:
# Calculate the length of each contour segment and return the cumulative contour length 
def cumulative_contour_length(contour):
    contour_length = np.zeros((len(contour),), dtype=float)
    contour_length[1:] = np.cumsum(np.sqrt(np.sum(np.square(contour[:-1, :] - contour[1:, :]),
                                                  axis=1)))
    return contour_length


# Take the length of a contour, map it to [0...1] and pick a point on the contour at the given ratio
def point_at_contour_ratio(contour, length_ratio, desired_ratio):
    # Pick the closest contour point after the given ratio
    idx = np.where(length_ratio >= desired_ratio)[0][0]
    if length_ratio[idx] == desired_ratio:
        # There is a precise contour point which is located at the desired ratio
        return contour[idx, :]
    
    # Find a point in between two contour points that lies at the desired point
    rest_ratio = desired_ratio - length_ratio[idx]
    rest_ratio_weigth = rest_ratio / (length_ratio[idx] - length_ratio[idx - 1])
    return (1 - rest_ratio_weigth) * contour[idx - 1, :] + rest_ratio_weigth * contour[idx, :]


# Extract leaf vein based on the leaf contour.
# "top_idx" represents the contour index of the top of the leaf
def extract_leaf_vein(leaf_contour, top_idx):
    # Compute left and right contours and the cumulative distances along each one,
    # as well as ratio values of each point along the contour it belongs to.
    left_contour = leaf_contour[:(1 + top_idx), :]
    right_contour = np.roll(leaf_contour, -1, axis=0)[(top_idx - 1):, :][::-1, :]  # Reverse order in order to start from petiole
    left_cum_length = cumulative_contour_length(left_contour)
    right_cum_length = cumulative_contour_length(right_contour)
    left_ratio = left_cum_length/left_cum_length[-1]
    right_ratio = right_cum_length/right_cum_length[-1] #  [x/right_cum_length[-1] for x in right_cum_length]

    # Pick the number of desired points along the leaf vein
    #     n_pts = max(left_contour.shape[0], right_contour.shape[0])
    n_pts = 20
    
    # Create leaf vein points as halfway points between points with the same ratio value on left and right contours
    leaf_vein = np.zeros((n_pts, 2), dtype=float)
    for i_pt in range(n_pts):
        curr_ratio = i_pt / (n_pts - 1)
        
        # Leaf vein points are calculated as halfway through the distance between
        # the points on left/right contours located at the same relative ratio
        leaf_vein[i_pt, :] = (point_at_contour_ratio(left_contour, left_ratio, curr_ratio) 
                              + point_at_contour_ratio(right_contour, right_ratio, curr_ratio)) / 2.0
    return leaf_vein


# Calculate a moving average along a given data vector "a"
# If radius is 2 the moving window has 1+2*radius size (1 for the middle element) 
def moving_average(a, radius):
    # w_sz is the window size
    w_sz = 1 + 2 * radius
    avg_res = np.zeros_like(a, dtype=float)
    
    # Compute cumulative sum
    a_cumsum = np.zeros((1 + a.size), dtype=float)  # One artificial zero is added at the beginning
    a_cumsum[1:] = np.cumsum(a, dtype=float)
    
    # Most average values can be computed as differences between the cumulative sum values at locations
    # found at w_sz distance from each other
    avg_res[radius:(a.size - radius)] = (a_cumsum[w_sz:] - a_cumsum[:(a.size - w_sz + 1)]) / w_sz
    
    # Some values at the beginning and at the end need special attention
    avg_res[0] = a[0]
    avg_res[-1] = a[-1]
    for idx in range(1, radius):
        curr_w_sz = 1 + 2 * idx
        avg_res[idx] = a_cumsum[curr_w_sz] / curr_w_sz
        avg_res[a.size - idx - 1] = (a_cumsum[-1] - a_cumsum[a.size - curr_w_sz]) / curr_w_sz        
    return avg_res


# Compute distance from fiven point to every point on the contour
def compute_contour_distances(point, contour):
    return cdist(point, contour).flatten()


In [132]:
# Compute width of a leaf lobe as widest diameter perpendicular to the leaf vein
def compute_width_of_leaf_lobe(leaf_mask_contour_XY, plant_center_XY):
    # Find leaf lobe's closest point to the plant center and make lobe contour start at that point
    leaf_dists = compute_contour_distances(plant_center_XY, leaf_mask_contour_XY)
    idx_bottom_pt = np.argmin(leaf_dists)
    rolled_leaf_contour_XY = np.roll(leaf_mask_contour_XY, -idx_bottom_pt, axis = 0)

    # Calculate initial leaf vein and smoothen it
    idx_top_pt = (np.argmax(leaf_dists) - idx_bottom_pt) % leaf_dists.shape[0]
    leaf_vein_initial = extract_leaf_vein(rolled_leaf_contour_XY, idx_top_pt)
    w_radius = max(3, int(np.round((leaf_vein_initial.shape[0] - 1) / 20.0)))
    leaf_vein = np.zeros_like(leaf_vein_initial, dtype=float)
    leaf_vein[:, 0] = moving_average(leaf_vein_initial[:, 0], w_radius)
    leaf_vein[:, 1] = moving_average(leaf_vein_initial[:, 1], w_radius)

    # Find widest leaf lobe diameter
    inters_pts_left = []
    inters_pts_right = []
    dists = [0.0]
    # First and last points on the leaf vein do not count
    for seg_idx_vein in range(1, leaf_vein.shape[0] - 1):
        
        # Compute diameter perpendicular to leaf vein at each point on the leaf vein
        curr_angle = np.arctan2(leaf_vein[seg_idx_vein, 1] - leaf_vein[(seg_idx_vein - 1), 1], 
                                leaf_vein[seg_idx_vein, 0] - leaf_vein[(seg_idx_vein - 1), 0])
        ortho_angle = np.mod(np.pi/2.0 + curr_angle, np.pi)
        line = leaf_vein[(seg_idx_vein - 1):(seg_idx_vein + 1), :][::-1, :].copy()
        line[1, 0] = line[0, 0] + 10 * np.cos(ortho_angle)
        line[1, 1] = line[0, 1] + 10 * np.sin(ortho_angle)
        curr_inters_pt_left = line_contour_intersection(line, rolled_leaf_contour_XY[:(1+idx_top_pt), :])
        curr_inters_pt_right = line_contour_intersection(line, rolled_leaf_contour_XY[idx_top_pt:, :])
        if curr_inters_pt_left is not None and curr_inters_pt_right is not None:
            inters_pts_left.append(curr_inters_pt_left)
            inters_pts_right.append(curr_inters_pt_right)
            dists.append(cdist(curr_inters_pt_left, curr_inters_pt_right)[0, 0])
    dists.append(0.0)
    
    # Select diameter with largest length
    idx_max_width = np.argmax(dists)
    
    # Return diameter's extremity points and leaf vein
    return inters_pts_left[idx_max_width], inters_pts_right[idx_max_width], leaf_vein


In [274]:
# Plot leaf data for the given excel file
def plot_all_leaf_data(dataset, excel_filepath, colors):
    
    # Read data
    path, excel_filename = split(excel_filepath)
    df_excel_plant_centers = pd.read_excel(join(path, 'plant_centers.xlsx'), sheet_name='plant_centers')
    
    # Get plant centers
    plant_centers = dict()
    for index, row in df_excel_plant_centers.iterrows():
        curr_date = row['Date']
        time_parts = row['Time'].split(':')
        curr_hour = int(time_parts[0])
        curr_min = int(time_parts[1])
        plant_centers[curr_date.year, curr_date.month, curr_date.day, curr_hour, curr_min] = \
            (row['X'], row['Y'])
    
    # Initialize useful data structures
    leaf_DAS = defaultdict(list)
    leaf_area = defaultdict(list)
    leaf_length = defaultdict(list)
    leaf_perimeter = defaultdict(list)
    leaf_width = defaultdict(list)
    leaf_circumference = defaultdict(list) 
    
    # Cycle through all leaf folders
    for leaf_folder in get_all_folders(join(path, 'Masks'), only_foldernames=True, sorted_list=False):
        
        # Create output folders
        makedirs(join(path, 'leaf_traits', leaf_folder, 'lobe_mask'), exist_ok=True)
        makedirs(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_contour'), exist_ok=True)
        makedirs(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_width'), exist_ok=True)
        makedirs(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_leaf_length'), exist_ok=True)
        makedirs(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_bounding_circle'), exist_ok=True)
        
        leaf_num = int(leaf_folder[-3:])
        # Cycle through all masks of the current leaf
        for filename in get_all_files(join(path, 'Masks', leaf_folder, 'hidden leaf mask seq'),
                                      only_filenames=True, sorted_list=False):
            
            # Get only essential parts of the current filename
            fileparts = [int(x) for x in filename.split('_')[1:8]]
            if (fileparts[2], fileparts[3], fileparts[4], fileparts[5], fileparts[6]) not in plant_centers.keys():
                # Skip lobe masks that do not correspond to any existing plant center
                continue
            
            # Read current leaf mask
            curr_mask = read_mask(join(path, 'Masks', leaf_folder, 'hidden leaf mask seq', filename))
            
            # Read current stem mask
            if not exists(join(path, 'Masks', leaf_folder, 'hidden stem mask seq', filename)):
                # Skip lobe masks that do not correspond to any existing stem masks
                continue
            curr_stem_mask = read_mask(join(path, 'Masks', leaf_folder, 'hidden stem mask seq', filename))
            
            ###############
            # Output Lobe #
            ###############
            out_img = np.zeros((curr_mask.shape[0], curr_mask.shape[1], 3), dtype=int)
            out_img[:, :, 0][curr_mask > 0] = 253
            out_img[:, :, 1][curr_mask > 0] = 231
            out_img[:, :, 2][curr_mask > 0] = 36
            cv2.imwrite(join(path, 'leaf_traits', leaf_folder, 'lobe_mask', filename),
                        out_img[:, :, ::-1])
            
            #######################
            # Output Lobe Contour #
            #######################
            curr_contour_float_YX = find_contours(curr_mask)[0]
            curr_contour_YX = np.round(curr_contour_float_YX).astype(int)
            out_img = np.stack((curr_mask, curr_mask.copy(), curr_mask.copy()), axis=2)
            contour_img = np.zeros(out_img.shape[:2], out_img.dtype)
            contour_img[curr_contour_YX[:, 0], curr_contour_YX[:, 1]] = 1
            temp_img = dilation(contour_img, square(3))
            temp_img_sel = temp_img > 0
            out_img[:, :, 0][temp_img_sel] = 0
            out_img[:, :, 1][temp_img_sel] = 191
            out_img[:, :, 2][temp_img_sel] = 191
            cv2.imwrite(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_contour', filename),
                        out_img[:, :, ::-1])
            
            ######################
            # Output Leaf Length #
            ######################
            full_mask = np.maximum(curr_mask, curr_stem_mask)
            curr_full_contour_float_YX = find_contours(full_mask)[0]
            curr_full_contour_YX = np.round(curr_full_contour_float_YX).astype(int)
            out_img = np.stack((full_mask, full_mask.copy(), full_mask.copy()), axis=2)
            temp_img = np.zeros(out_img.shape[:2], out_img.dtype)
            curr_plant_center_XY = np.reshape(
                np.array(plant_centers[fileparts[2], fileparts[3], fileparts[4], fileparts[5], fileparts[6]],
                         dtype=float),
                [1, 2])
            stem_coords = np.transpose(np.nonzero(curr_stem_mask))
            stem_dists = []
            for i_stem in range(stem_coords.shape[0]):
                stem_dists.append(np.min(compute_contour_distances(stem_coords[i_stem:(i_stem + 1), :],
                                                                   curr_contour_YX)))
            idx_stem_closest_to_contour = np.argmin(stem_dists)
            max_width_pt_left, max_width_pt_right, curr_leaf_vein_XY_float = compute_width_of_leaf_lobe(
                curr_contour_YX[:, ::-1],
                stem_coords[idx_stem_closest_to_contour:(idx_stem_closest_to_contour + 1), ::-1])
            curr_leaf_vein_XY = np.round(curr_leaf_vein_XY_float).astype(int)
            for i_vein in range(curr_leaf_vein_XY.shape[0] - 1):
                seg_start = curr_leaf_vein_XY[i_vein, :]
                seg_end = curr_leaf_vein_XY[i_vein + 1, :]
                seg_y, seg_x = line(int(seg_start[1]), int(seg_start[0]),
                                    int(seg_end[1]), int(seg_end[0]))
                temp_img[seg_y, seg_x] = 1
            stem2 = curr_stem_mask.copy()
            stem2[curr_mask > 0] = 0
            temp_img_sel = np.logical_or(temp_img > 0, stem2 > 0)
            temp_img_2 = np.zeros_like(temp_img)
            temp_img_2[temp_img_sel] = 1
            temp_img_2 = dilation(temp_img_2, square(2))
            temp_img_sel_2 = temp_img_2 > 0
            out_img[:, :, 0][temp_img_sel_2] = 255
            out_img[:, :, 1][temp_img_sel_2] = 0
            out_img[:, :, 2][temp_img_sel_2] = 0
            cv2.imwrite(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_leaf_length', filename),
                        out_img[:, :, ::-1])
            
            #####################
            # Output Lobe Width #
            #####################
            width_mask_y, width_mask_x = line(int(max_width_pt_left[0, 1]), int(max_width_pt_left[0, 0]),
                                              int(max_width_pt_right[0, 1]), int(max_width_pt_right[0, 0]))
            out_img = np.stack((curr_mask, curr_mask.copy(), curr_mask.copy()), axis=2)
            temp_img = np.zeros(out_img.shape[:2], out_img.dtype)
            temp_img[width_mask_y, width_mask_x] = 1
            temp_img = dilation(temp_img, square(2))
            temp_img_sel = temp_img > 0
            out_img[:, :, 0][temp_img_sel] = 0
            out_img[:, :, 1][temp_img_sel] = 175
            out_img[:, :, 2][temp_img_sel] = 0
            cv2.imwrite(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_width', filename),
                        out_img[:, :, ::-1])

            ##########################
            # Output Bounding Circle #
            ##########################
            out_img = np.stack((curr_mask, curr_mask.copy(), curr_mask.copy()), axis=2)
            temp_img = np.zeros(out_img.shape[:2], out_img.dtype)
            contour_dists = []
            dist_pairs = []
            for i_pix in range(curr_contour_YX.shape[0] - 1):
                curr_dists = compute_contour_distances(curr_contour_YX[i_pix:(i_pix + 1), :],
                                                       curr_contour_YX[(i_pix + 1):, :])
                idx_max_dist = np.argmax(curr_dists)
                contour_dists.append(curr_dists[idx_max_dist])
                dist_pairs.append((i_pix, i_pix + 1 + idx_max_dist))
            idx_best_pair = np.argmax(contour_dists)
            center_Y = (curr_contour_YX[dist_pairs[idx_best_pair][0], 0]
                        + curr_contour_YX[dist_pairs[idx_best_pair][1], 0]) / 2.0
            center_X = (curr_contour_YX[dist_pairs[idx_best_pair][0], 1]
                        + curr_contour_YX[dist_pairs[idx_best_pair][1], 1]) / 2.0
            circle_coords_Y, circle_coords_X = ellipse_perimeter(
                int(round(center_Y)),
                int(round(center_X)),
                int(round(contour_dists[idx_best_pair] / 2.0)),
                int(round(contour_dists[idx_best_pair] / 2.0)))
            sel_idx = np.logical_or(circle_coords_Y < temp_img.shape[0],
                                    circle_coords_X < temp_img.shape[1])
            temp_img[circle_coords_Y[sel_idx], circle_coords_X[sel_idx]] = 1
            temp_img = dilation(temp_img, square(2))
            temp_img_sel = temp_img > 0
            out_img[:, :, 0][temp_img_sel] = 0
            out_img[:, :, 1][temp_img_sel] = 0
            out_img[:, :, 2][temp_img_sel] = 255
            cv2.imwrite(join(path, 'leaf_traits', leaf_folder, 'lobe_mask_with_bounding_circle', filename),
                        out_img[:, :, ::-1])
            #####################
    

In [ ]:
# List of replicates to plot.
# First element is dataset number.
# Second element is path to excel file of the given replicate
excel_filepaths = [
    (1, r'D:\Path\to\excel\files\of\chosen\replicate\rep_xx.xlsx'),
    (1, r'D:\Path\to\excel\files\of\chosen\replicate\rep_yy.xlsx'),
    (2, r'D:\Path\to\excel\files\of\chosen\replicate\rep_zz.xlsx'),
    (2, r'D:\Path\to\excel\files\of\chosen\replicate\rep_tt.xlsx')
]

# Create palette with distinct colors
init_colors = distinctipy.get_colors(75, list(combinations_with_replacement([1, 0.9, 0.8], 3)))
colors = [x for x in init_colors if np.sum(sorted(x, reverse=True)[:2]) < 1.5]
colors.insert(4, (1, 0.68, 0))

# Start plotting
for ds, ep in excel_filepaths:
    print(ep)
    plot_all_leaf_data(ds, ep, colors)
